In [1]:
!pwd

/notebooks/notebooks/SnowparkRnD/UDFs


In [3]:
from Run_Experiment import run_experiment

In [4]:
import os
from snowflake.snowpark.functions import udf

In [ ]:
connection_parameters = {
    "account": "ug94937.us-east4.gcp",
    "user": "ADITYASINGH",
    "password": os.environ(),
    "role": "ADITYASINGH",  # optional
    "warehouse": "FOSFOR_INSIGHT_WH",  # optional
    "database": "FIRST_DB",  # optional
    "schema": "PUBLIC",  # optional
    }
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

In [5]:
session.add_import("tests/resources/test_udf_dir/test_udf_file.py", import_path="resources.test_udf_dir.test_udf_file")
session.add_packages("numpy")

NameError: name 'session' is not defined